# final_05_2025.ipynb

Building cohort, data processing, classification, # of clusters, latent space extraction

In [1]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable


In [68]:
import pandas as pd
import numpy as np
import re
import time
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import seaborn as sns

import umap.umap_ as umap
import umap.plot

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score, roc_curve, auc
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, silhouette_score, davies_bouldin_score, normalized_mutual_info_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, normalize, label_binarize
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import HDBSCAN, KMeans, DBSCAN, SpectralClustering, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA

from scipy.stats import entropy
from scipy.cluster.hierarchy import fcluster, linkage, dendrogram
from scipy.sparse import csr_matrix

from yellowbrick.cluster import KElbowVisualizer

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import random
import warnings

In [69]:
mimic4_path = '/global/cfs/projectdirs/m1532/Projects_MVP/_datasets/MIMIC_IV/physionet.org/files/mimiciv/2.2/'
root_path = '/global/cfs/projectdirs/m1532/Projects_MVP/Sophia/2024/'

## util

In [70]:
def classify(race):
    if 'BLACK' in race:
        return 'B'
    elif 'WHITE' in race:
        return 'W'
    else:
        return 'O'

In [71]:
def assign_group(row):
    if (row['osa'] == True)and (row['hf'] == False):
        return 0 # OSA only
    elif (row['osa'] == True)and (row['hf'] == True):
        return 1 # OSA & HF
    elif (row['osa'] == False)and (row['hf'] == True):
        return 2 # HF only

In [72]:
#- can use group instead.
def group_str(group_int):
    if group_int == 0:
        return 'OSA Only'
    elif group_int == 1:
        return 'OSA & HF'
    else:
        return 'HF Only' 

In [73]:
#- need to adjust based on cohort.
def read_cohort_without_id(file):
    cohort_df = pd.read_csv(file).drop(columns = ['Unnamed: 0'], errors='ignore') 
    cohort_df['group_name'] = cohort_df['group'].apply(group_str)
    # Group counts
    group_counts = cohort_df['group_name'].value_counts()
    hf_count = group_counts['HF Only']
    osa_count = group_counts['OSA Only']

    # Downsample "HF Only" group
    hf_df = cohort_df[cohort_df['group_name'] == 'HF Only']
    osa_df = cohort_df[cohort_df['group_name'] == 'OSA Only']
    osa_hf_df = cohort_df[cohort_df['group_name'] == 'OSA & HF']

    # Downsample "HF Only" to the same size as "OSA Only"
    hf_df_downsampled = hf_df.sample(n=osa_count, random_state=42)

    # Combine the downsampled "HF Only" group with the other groups
    downsampled_cohort_df = pd.concat([hf_df_downsampled, osa_df, osa_hf_df])
    X = downsampled_cohort_df.drop(columns = ['subject_id', 'hadm_id', 'group', 'group_name'], errors='ignore')
    Y = downsampled_cohort_df['group']
    
    return X,Y

In [74]:
#- need to adjust based on cohort.
def read_cohort_with_id(cohort_df):
    # cohort_df = pd.read_csv(file).drop(columns = ['Unnamed: 0'], errors='ignore') 
    cohort_df['group_name'] = cohort_df['group'].apply(group_str)
    # Group counts
    group_counts = cohort_df['group_name'].value_counts()
    hf_count = group_counts['HF Only']
    osa_count = group_counts['OSA Only']

    # Downsample "HF Only" group
    hf_df = cohort_df[cohort_df['group_name'] == 'HF Only']
    osa_df = cohort_df[cohort_df['group_name'] == 'OSA Only']
    osa_hf_df = cohort_df[cohort_df['group_name'] == 'OSA & HF']

    # Downsample "HF Only" to the same size as "OSA Only"
    hf_df_downsampled = hf_df.sample(n=osa_count, random_state=42)

    # Combine the downsampled "HF Only" group with the other groups
    downsampled_cohort_df = pd.concat([hf_df_downsampled, osa_df, osa_hf_df])
    X = downsampled_cohort_df
    Y = downsampled_cohort_df['group']
    
    return X,Y

In [75]:
def one_hot_encode(df, one_hot_cols, drop_id=True):
    # Initialize the one-hot encoder
    one_hot_encoder = OneHotEncoder(drop='first', sparse_output=False)
    # Fit and transform the columns with the encoder
    one_hot_encoded_data = one_hot_encoder.fit_transform(df[one_hot_cols])
    # Convert the result into a DataFrame with appropriate column names
    one_hot_df = pd.DataFrame(one_hot_encoded_data, columns=one_hot_encoder.get_feature_names_out(one_hot_cols))
    # Drop the original nominal columns from the dataset
    data = df.drop(one_hot_cols, axis=1)
    
    # Concatenate the one-hot encoded columns with the original dataset
    data.reset_index(drop=True, inplace=True)
    one_hot_df.reset_index(drop=True, inplace=True)
    data_1 = pd.concat([data, one_hot_df], axis=1)

    # Display the first few rows of the dataset after one-hot encoding
    if drop_id:
        data_final = data_1.drop(['subject_id', 'hadm_id', 'osa', 'hf', 'text', 'group_name'], axis = 1)
    else:
        data_final = data_1.drop(['osa', 'hf', 'text', 'group_name'], axis = 1)
    return data_final                      

In [76]:
def train_models(X, Y):
    models = {
        'Logistic Regression': LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', C=0.2, max_iter=5000),
        'Random Forest': RandomForestClassifier(n_estimators=50),
        #-'Support Vector Machine': SVC(probability=True)
    }
    
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
    
    results = {}
    
    for model_name, model in models.items():
        print(f"Training {model_name}")
        start_time = time.time()
        scores = cross_val_score(model, X, Y, scoring='roc_auc_ovr_weighted', cv=cv, n_jobs=128, verbose=True)
        end_time = time.time()
        
        # Store results
        results[model_name] = {
            'mean_score': np.mean(scores),
            'std_score': np.std(scores),
            'runtime': end_time - start_time
        }
        
        print(f'{model_name} - KFold-Results: Mean ROC AUC: {results[model_name]["mean_score"]:.3f} (Std: {results[model_name]["std_score"]:.3f})')
        print(f"{model_name} - Runtime: {results[model_name]['runtime']:.6f} seconds\n")
        
    rf_model = models.get('Random Forest')
    
    for train_index, test_index in cv.split(X, Y):
        rf_model=rf_model.fit(X.iloc[train_index], Y.iloc[train_index])
        y_train_preds = rf_model.predict(X.iloc[train_index])
        y_test_preds = rf_model.predict(X.iloc[test_index])

        print("training results")
        print(classification_report(Y.iloc[train_index].values, y_train_preds))
        print(confusion_matrix(Y.iloc[train_index].values, y_train_preds))
        #print("roc_auc_score:", roc_auc_score(Y.iloc[train_index].values, y_train_preds, multi_class = 'ovr'))
        #print("f1_score:", f1_score(Y.iloc[train_index].values, y_train_preds))
        print("testing results")
        print(classification_report(Y.iloc[test_index].values, y_test_preds))
        print(confusion_matrix(Y.iloc[test_index].values, y_test_preds))
        #print("roc_auc_score:", roc_auc_score(Y.iloc[test_index].values, y_test_preds, multi_class = 'ovr'))
        #print("f1_score:",f1_score(Y.iloc[test_index].values, y_test_preds))

        break
    
    return results, rf_model

In [77]:
# labels should be 
def extract_n_feature_importance(X, rf_model, num_features=25, labels={}, specified_feature_list=[]):
        """
    Extract and visualize the top `n` most important features from a trained 
    Random Forest model using the Mean Decrease in Impurity (MDI) method.

    Parameters:
    -----------
    X : pandas.DataFrame
        The feature matrix used for training the Random Forest model. 
        Columns represent the features.
        
    rf_model : sklearn.ensemble.RandomForestClassifier or RandomForestRegressor
        The trained Random Forest model from which to extract feature importances.
        
    num_features : int, optional, default=25
        The number of top features to display based on their importance. If `num_features`
        exceeds the number of features in X, the function will display all features.
        
    labels : dict, optional, default={}
        A dictionary to map feature names to more descriptive labels in the output. 
        If provided, the keys should be feature names from `X.columns`, and the values 
        should be the corresponding labels to use in the plot.
        
    specified_feature_list : list, optional, default=[]
        A list of specific features to highlight in the visualization. 
        If provided, these features will be emphasized in the plot.

    Returns:
    --------
    all_feature_importance_count_dict : dict
        A dictionary containing the importance values for all features in `X`. 
        The keys are the feature names, and the values are the importance scores.
    
    Visual Output:
    --------------
    A bar plot showing the top `n` feature importances, with the corresponding 
    standard deviation bars if available. The plot title and labels are customized 
    based on the input parameters.

    Notes:
    ------
    - The function sorts the features by their importance in descending order and 
      displays the top `num_features`.
    - If `labels` are provided, the feature names will be replaced with the specified labels.
    - This function is intended for use with Random Forest models trained using 
      scikit-learn's `RandomForestClassifier` or `RandomForestRegressor`.
    
    Example Usage:
    --------------
    >>> feature_importance_dict = extract_n_feature_importance(X, rf_model, num_features=30, labels=feature_labels)
    """
    
    
        importances = rf_model.feature_importances_
        std = np.std([tree.feature_importances_ for tree in rf_model.estimators_], axis=0)

        # print(importances.shape)

        #- Sort the feature importance in descending order
        sorted_indices = np.argsort(importances)[::-1]

        feature_labels=X.columns

        #- Display the top 25 important features.
        feature_importance_count_dict  = {}
        for f in range(num_features):
            feature_importance_count_dict[feature_labels[sorted_indices[f]]] = importances[sorted_indices[f]]
            # print(f, feature_labels[sorted_indices[f]])

        std = std[:num_features]
        std.shape

        feature_importances_count = pd.Series(feature_importance_count_dict).rename(labels)
        feature_importances_count.shape

        all_feature_importance_count_dict  = {}
        for f in range(len(importances)):
            all_feature_importance_count_dict[feature_labels[sorted_indices[f]]] = importances[sorted_indices[f]]
            # print(f, feature_labels[sorted_indices[f]])

        for label, value in feature_importances_count.items():
            print(f"{label}: {value}")

        #fig, ax = plt.subplots()
        fig = plt.figure()
        ax = fig.add_axes([0,0,2,1])
        feature_importances_count.plot.bar(y=std, ax=ax, width=.8)
        ax.set_title("Feature importances using MDI")
        ax.set_ylabel("Mean decrease in impurity")
        # fig.tight_layout()

        return all_feature_importance_count_dict

In [78]:
def extract_specified_feature_importance(all_feature_importance_count_dict={}, feature_list=[], num_features=50, labels={}):
    specified_feature_importance = {}
    for k, v in all_feature_importance_count_dict.items():
        if k in feature_list:
            specified_feature_importance[k] = v
        
    print(specified_feature_importance)
    
    specified_feature_importance_count = pd.Series(specified_feature_importance).rename(labels)
    specified_feature_importance_count.shape
    
    fig = plt.figure()
    ax = fig.add_axes([0,0,2,1])
    specified_feature_importance_count.plot.bar(y=std, ax=ax, width=.8)
    ax.set_title("Feature importances")
    ax.set_ylabel("Mean decrease in impurity")
    # fig.tight_layout()

In [79]:
def umap_embedding(X, n_neighbors=15, min_dist=0.1, n_components=2, random_state=42):
    """
    Perform UMAP embedding on the dataset X with the provided parameters.

    Parameters:
    - X: The feature dataset to be transformed.
    - n_neighbors: The size of local neighborhood (in terms of number of neighboring sample points) used for manifold approximation.
    - min_dist: The minimum distance between points in the low-dimensional space.
    - n_components: The number of dimensions of the low-dimensional space.
    - random_state: The seed used by the random number generator.

    Returns:
    - clusterable_embedding: The transformed feature dataset.
    - runtime: The time taken to perform the UMAP embedding.
    """
    start_time = time.time()
    clusterable_embedding = umap.UMAP(
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        n_components=n_components,
        random_state=random_state
    ).fit_transform(X)
    end_time = time.time()

    # Calculate the runtime
    runtime = end_time - start_time
    print(f"UMAP Runtime: {runtime:.6f} seconds")
    
    return clusterable_embedding, runtime

In [80]:
def plot_umap_embedding(clusterable_embedding, labels, group, colors=None, n_clusters=None, title='UMAP Embedding', xlabel='Embedding-1', ylabel='Embedding-2', background_color='white', point_size=0.1, point_alpha=0.5):
    """
    Creates a scatter plot of UMAP embeddings.

    Parameters:
    - clusterable_embedding: The transformed feature dataset.
    - labels: The cluster labels for each data point.
    - group: The unique number for each group.
    - colors: A list of colors to use for the clusters.
    - n_clusters: The number of clusters.
    - title: The title of the plot.
    - xlabel: The label for the x-axis.
    - ylabel: The label for the y-axis.
    - background_color: The background color of the plot.
    - point_size: The size of the points in the scatter plot.
    - point_alpha: The alpha transparency for the points.
    """
    if colors is None:
        colors = ['orange', 'green', 'blue']  # Default colors if none provided
    if n_clusters is None:
        n_clusters = len(group)
    
    cmap = mcolors.ListedColormap(colors[:n_clusters])
    
    fig = plt.figure(figsize=(8, 6))
    ax = plt.gca()
    ax.set_facecolor(background_color)
    
    sc = plt.scatter(clusterable_embedding[:, 0], clusterable_embedding[:, 1], c=labels, s=point_size, alpha=point_alpha, cmap=cmap)
    
    labels_group_str = labels.apply(group_str)
    
    cbar = fig.colorbar(sc)
    cbar.set_ticks(group)
    cbar.set_ticklabels(labels_group_str.unique())
    
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

### PCA

In [81]:
#-
def apply_pca_with_n_components(data, n_components):
    """
    Applies PCA to the given data with n number of components.

    Parameters:
    - data: ndarray of shape (n_samples, n_features) The input data to perform PCA on.
    
    Returns:
    - pca_data: ndarray of shape (n_samples, n_components) The transformed data with the selected PCA components.
    """

    
    # Apply PCA with the selected number of components
    pca = PCA(n_components=n_components)
    pca_data = pca.fit_transform(data)
    
    return pca_data

In [82]:
#-
def train_models_results(X, Y):
    models = {
        'Logistic Regression': LogisticRegression(solver='lbfgs', penalty='l2', C=0.2, max_iter=5000),
        'Random Forest': RandomForestClassifier(n_estimators=50),
        #-'Support Vector Machine': SVC(probability=True)
    }
    
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
    
    results = {}
    
    for model_name, model in models.items():
        print(f"Training {model_name}")
        start_time = time.time()
        scores = cross_val_score(model, X, Y, scoring='roc_auc_ovr_weighted', cv=cv, n_jobs=128, verbose=True)
        end_time = time.time()
        
        # Store results
        results[model_name] = {
            'mean_score': np.mean(scores),
            'std_score': np.std(scores),
            'runtime': end_time - start_time
        }
        
        print(f'{model_name} - KFold-Results: Mean ROC AUC: {results[model_name]["mean_score"]:.3f} (Std: {results[model_name]["std_score"]:.3f})')
        print(f"{model_name} - Runtime: {results[model_name]['runtime']:.6f} seconds\n")
    
    return results[model_name]["mean_score"]

In [83]:
#-
def score_pca(X, Y, n_components):
    """
    Get wAUC score of RF model using PCA

    Parameters:
    - X: ndarray of shape (n_samples, n_features) The input data to perform PCA on.
    - Y: The labels.
    - n_components: int The number of components selected

    Returns:
    - score: float Average reconstruction error of RF model using PCA
    """
    
    start_time = time.time()
    # Apply PCA
    pca_data = apply_pca_with_n_components(X, n_components)
    end_time = time.time()
    print("Runtime for PCA: ", end_time - start_time)

    #-
    score = train_models_results(pca_data, Y)

    return score, pca_data

### Autoencoder

In [84]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU
    torch.backends.cudnn.deterministic = True  # Make sure CUDA operations are deterministic
    torch.backends.cudnn.benchmark = False  # Disable CUDA's auto-tuner that finds the best algorithm to use for your hardware

In [85]:
#- sparse data
class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dims):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 400),
            nn.BatchNorm1d(400),
            nn.LeakyReLU(0.2),  # LeakyReLU
            nn.Linear(400, 200),
            nn.BatchNorm1d(200),
            nn.LeakyReLU(0.2),
            nn.Linear(200, 100),
            nn.BatchNorm1d(100),
            nn.LeakyReLU(0.2),
            nn.Linear(100, encoding_dims),
            nn.BatchNorm1d(encoding_dims),
            nn.LeakyReLU(0.2)
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dims, 100),
            nn.BatchNorm1d(100),
            nn.LeakyReLU(0.2),
            nn.Linear(100, 200),
            nn.BatchNorm1d(200),
            nn.LeakyReLU(0.2),
            nn.Linear(200, 400),
            nn.BatchNorm1d(400),
            nn.LeakyReLU(0.2),
            nn.Linear(400, input_dim),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

    def encode(self, x):
        return self.encoder(x)
    
    #- for DAE only
    def add_noise(self, x, noise_factor=0.3):
        noisy = x + noise_factor * torch.randn_like(x)
        noisy = torch.clip(noisy, 0., 1.)
        return noisy

### Clustering

In [86]:
def plot_and_return_optimal_clusters(data, k_range=(4, 30), metrics=['silhouette', 'calinski_harabasz']):
    """
    Plot multiple elbow plots in subplots for different evaluation metrics with K-means clustering.
    Returns the optimal number of clusters for each metric.

    Parameters:
    data (array-like): The input data to cluster.
    k_range (tuple): Range of number of clusters to test. Default is (2, 20).
    metrics (list): List of metrics to use for evaluation. Options include 'silhouette', 'calinski_harabasz'
                    Default is ['silhouette', 'calinski_harabasz', 'davies_bouldin'].

    Returns:
    dict: A dictionary containing the optimal number of clusters for each metric.
    """
    fig, axes = plt.subplots(len(metrics), 1, figsize=(10, 6 * len(metrics)))
    optimal_clusters = {}


    for i, metric in enumerate(metrics, start=0):
        if metric == 'silhouette' or metric == 'calinski_harabasz':
            model = KMeans()
            visualizer = KElbowVisualizer(model, k=k_range, metric=metric, timings=False, ax=axes[i])
            visualizer.fit(data)
            visualizer.finalize()
            optimal_clusters[metric] = visualizer.elbow_value_
        else:
            print(f"Warning: Metric '{metric}' is not supported for KElbowVisualizer. Skipping.")
    
    plt.tight_layout()
    plt.show()

    return optimal_clusters

In [87]:
def get_kmeans_davies_bouldin_score(data, center):
    '''
    returns the kmeans score regarding Davies Bouldin for points to centers
    INPUT:
        data - the dataset you want to fit kmeans to
        center - the number of centers you want (the k value)
    OUTPUT:
        score - the Davies Bouldin score for the kmeans model fit to the data
    '''
    #instantiate kmeans
    kmeans = KMeans(n_clusters=center)
    # Then fit the model to your data using the fit method
    model = kmeans.fit_predict(data)
    
    # Calculate Davies Bouldin score
    score = davies_bouldin_score(data, model)
    
    return score

In [88]:
def  get_kmeans_gap_statistic(data, nrefs=3, maxClusters=20):
    """
    Calculates KMeans optimal K using Gap Statistic 
    Params:
        data: ndarry of shape (n_samples, n_features)
        nrefs: number of sample reference datasets to create
        maxClusters: Maximum number of clusters to test for
    Returns: (gaps, optimalK)
    """
    gaps = np.zeros((len(range(1, maxClusters)),))
    resultsdf = pd.DataFrame({'clusterCount':[], 'gap':[]})
    for gap_index, k in enumerate(range(1, maxClusters)):
# Holder for reference dispersion results
        refDisps = np.zeros(nrefs)
# For n references, generate random sample and perform kmeans getting resulting dispersion of each loop
        for i in range(nrefs):
            
            # Create new random reference set
            randomReference = np.random.random_sample(size=data.shape)
            
            # Fit to it
            km = KMeans(k)
            km.fit(randomReference)
            
            refDisp = km.inertia_
            refDisps[i] = refDisp
            # Fit cluster to original data and create dispersion
        km = KMeans(k)
        km.fit(data)
        
        origDisp = km.inertia_
        # Calculate gap statistic
        gap = np.log(np.mean(refDisps)) - np.log(origDisp)
        # Assign this loop's gap statistic to gaps
        gaps[gap_index] = gap
        
        
        new_row = pd.DataFrame({'clusterCount': [k], 'gap': [gap]})

        # Use concat to concatenate the new row to resultsdf
        resultsdf = pd.concat([resultsdf, new_row], ignore_index=True)
    
        
        #resultsdf = resultsdf.append({'clusterCount':k, 'gap':gap}, ignore_index=True)
    return (gaps.argmax() + 1, resultsdf)


In [89]:
def evaluate_kmeans(data, n_clusters=3, random_state=42):
    """
    Perform K-means clustering on the given data and evaluate the clustering.

    Parameters:
    data (array-like): The input data to cluster.
    n_clusters (int): The number of clusters to form.
    plot_tsne (bool): Whether to plot the t-SNE visualization.
    random_state (int): The seed used by the random number generator.

    Returns:
    dict: A dictionary containing evaluation metrics.
    """
    # Fit K-means
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state).fit(data)
    labels = kmeans.labels_
    
    # Calculate metrics
    silhouette_avg = silhouette_score(data, labels)
    db_index = davies_bouldin_score(data, labels)
    wcss = kmeans.inertia_
    
    metrics = {
        'Silhouette Score': silhouette_avg, #--high Measures how similar each point is to its own cluster compared to other clusters.
        'Davies-Bouldin Index': db_index,  #- low Measures the average similarity ratio of each cluster with its most similar cluster.
        'WCSS': wcss                       #- low compactness
    }
    
    #print(f'Silhouette Score: {silhouette_avg}')
    #print(f'Davies-Bouldin Index: {db_index}')
    #print(f'Within-Cluster Sum of Squares (WCSS): {wcss}')
    
    return metrics

In [90]:
def evaluate_gmm(data, n_components=3, random_state=42):
    """
    Perform Gaussian Mixture Model (GMM) clustering on the given data and evaluate the clustering.

    Parameters:
    data (array-like): The input data to cluster.
    n_components (int): The number of Gaussian components.
    plot_tsne (bool): Whether to plot the t-SNE visualization.
    random_state (int): The seed used by the random number generator.

    Returns:
    dict: A dictionary containing evaluation metrics.
    """
    # Fit GMM
    gmm = GaussianMixture(n_components=n_components, random_state=random_state).fit(data)
    labels = gmm.predict(data)
    
    # Calculate metrics
    silhouette_avg = silhouette_score(data, labels)
    db_index = davies_bouldin_score(data, labels)
    bic = gmm.bic(data)
    
    metrics = {
        'Silhouette Score': silhouette_avg,
        'Davies-Bouldin Index': db_index,
        'BIC': bic
    }
    
    #print(f'Silhouette Score: {silhouette_avg}')
    #print(f'Davies-Bouldin Index: {db_index}')
    #print(f'Bayesian Information Criterion (BIC): {bic}')
    
    return metrics

In [91]:
def hierarchical_clustering(latent_representations, truth_labels, threshold=200, figsize=(20, 10), y_interval=50, p=10):
    """
    Performs hierarchical clustering on the given latent representations, plots the dendrogram,
    and returns the cluster labels along with performance metrics.

    Parameters:
    - latent_representations: ndarray of shape (n_samples, n_features) The latent representations obtained from the Autoencoder.
    - truth_labels: array-like of shape (n_samples,)  The true labels of the samples.
    - threshold: float, optional (default=200)   The distance threshold for forming clusters.
    - figsize: tuple, optional (default=(20, 10)) The size of the figure for plotting the dendrogram.
    - y_interval: int, optional (default=50) The interval for y-axis ticks on the dendrogram.

    Returns:
    - cluster_labels: ndarray of shape (n_samples,) The cluster labels for each sample.
    - ari: float The Adjusted Rand Index of the clustering.
    - nmi: float The Normalized Mutual Information of the clustering.
    """
    # Perform hierarchical clustering using Ward's method
    linked = linkage(latent_representations, 'ward')

    # Create and plot the dendrogram
    plt.figure(figsize=figsize)
    # Set custom y-axis intervals
    max_d = np.max(linked[:, 2])
    y_lim = max_d - (max_d/2)
    
    # Ensure y-ticks are within the range of the dendrogram
    if max_d < y_interval:
        y_ticks = np.arange(0, max_d + max_d/10, max_d/10)
        
    plt.yticks(np.arange(0, max_d + y_interval, y_interval))  # Adjust the interval as needed
    dendrogram(linked, p=p, truncate_mode='level')
    plt.xlabel('Sample')
    plt.ylabel('Distance')
    plt.ylim(0, y_lim)
    plt.title('Hierarchical Clustering Dendrogram')
    # Add a horizontal line at the threshold
    plt.axhline(y=threshold, color='r', linestyle='--')
    plt.show()

    # Get cluster labels based on the threshold
    cluster_labels = fcluster(linked, threshold, criterion='distance')

    # Calculate performance metrics
    ari = adjusted_rand_score(truth_labels, cluster_labels)
    nmi = normalized_mutual_info_score(truth_labels, cluster_labels)

    return linked, cluster_labels, ari, nmi

# I. BUILD COHORT
Build combined OSA and HF cohort with:

subject_id, hadm_id, gender, race, age, BMI, admission type, Charlson Comorbidity Index score, length of stay, hospital expiration flag, and insurance.

In [92]:
admissions = pd.read_csv(mimic4_path + 'hosp/admissions.csv')

In [93]:
patients = pd.read_csv(mimic4_path + 'hosp/patients.csv')

In [94]:
dx_codes = pd.read_csv(mimic4_path + 'hosp/diagnoses_icd.csv')

In [95]:
dx_description = pd.read_csv(mimic4_path + 'hosp/d_icd_diagnoses.csv')

In [96]:
omr = pd.read_csv(mimic4_path + 'hosp/omr.csv')

In [97]:
charlson = pd.read_csv('/global/cfs/projectdirs/m1532/Projects_MVP/Sophia/derived/charlson.csv')

In [98]:
notes = pd.read_csv('/global/cfs/projectdirs/m1532/Projects_MVP/_datasets/MIMIC_IV/physionet.org/files/mimic-iv-note/2.2/note/discharge.csv')

## Obstructive sleep apnea (OSA)
Extract admissions with OSA ICD diagnosis code.

In [99]:
osa_codes = pd.read_csv(root_path + 'icd_codes/osa.csv')

In [100]:
osa = osa_codes.merge(dx_description, on ='icd_code', how = 'inner')

In [1]:
osa = osa.merge(dx_codes, on='icd_code', how = 'inner')

NameError: name 'osa' is not defined

In [102]:
osa = osa.merge(admissions, on = ['hadm_id', 'subject_id'] , how ='inner')

In [103]:
osa = osa[['subject_id', 'hadm_id']]

In [104]:
osa['osa'] = True

In [105]:
print(osa.shape)
osa.head(5)

(33005, 3)


,subject_id,hadm_id,osa
0,13340824,26708815,True
1,10002013,24848509,True
2,10002167,29383904,True
3,10002221,21008195,True
4,10003019,20030125,True


In [106]:
osa.drop_duplicates(subset = ['subject_id', 'hadm_id'], inplace = True)
osa.reset_index(drop=True, inplace = True)

In [107]:
osa.shape

(32974, 3)

In [108]:
osa['subject_id'].unique().size

14915

## Heart failure (HF)
Extract admissions with HF ICD diagnosis code.

In [109]:
hf_codes = pd.read_csv(root_path + 'icd_codes/heart_failure.csv')

In [110]:
hf = hf_codes.merge(dx_description, on ='icd_code', how = 'inner')

In [111]:
hf = hf.merge(dx_codes, on='icd_code', how = 'inner')

In [112]:
hf = hf.merge(admissions, on = ['hadm_id', 'subject_id'] , how ='inner')

In [113]:
hf = hf[['subject_id', 'hadm_id']]

In [114]:
hf['hf'] = True

In [117]:
print(hf.shape)
hf.head(5)

(53024, 3)


,subject_id,hadm_id,hf
0,10224335,28418370,True
1,10543835,20482351,True
2,10624524,20456042,True
3,11032631,25353318,True
4,11034601,21104656,True


In [118]:
hf.drop_duplicates(subset = ['subject_id', 'hadm_id'], inplace = True)
hf.reset_index(drop=True, inplace = True)

In [119]:
hf['subject_id'].unique().size

21774

## >=1 admission vs. >=2 admissions

### OSA

In [120]:
print("# of patients with OSA: ", osa['subject_id'].unique().size)
print("Number of patients with OSA with ONLY 1 admission: ", osa.groupby("subject_id").filter(lambda x: len(x) == 1).shape[0])

osa_multiple_adm = osa.groupby("subject_id").filter(lambda x: len(x) > 1).reset_index(drop=True)
print("Number of patients with OSA with MORE THAN 1 admission: ", osa_multiple_adm['subject_id'].unique().size)

# of patients with OSA:  14915
Number of patients with OSA with ONLY 1 admission:  8952
Number of patients with OSA with MORE THAN 1 admission:  5963


In [121]:
print("Total # of OSA admissions where patient has >= 1 admission): ", osa['hadm_id'].unique().size)
print("Total # of OSA admissions where patient has >= 2 admissions): ", osa_multiple_adm['hadm_id'].unique().size)

Total # of OSA admissions where patient has >= 1 admission):  32974
Total # of OSA admissions where patient has >= 2 admissions):  24022


### HF

In [122]:
print("# of patients with HF: ", hf['subject_id'].unique().size)
print("Number of patients with HF with ONLY 1 admission: ", hf.groupby("subject_id").filter(lambda x: len(x) == 1).shape[0])

hf_multiple_adm = hf.groupby("subject_id").filter(lambda x: len(x) > 1).reset_index(drop=True)
print("Number of patients with HF with MORE THAN 1 admission: ", hf_multiple_adm['subject_id'].unique().size)

# of patients with HF:  21774
Number of patients with HF with ONLY 1 admission:  12296
Number of patients with HF with MORE THAN 1 admission:  9478


In [123]:
print("Total # of HF admissions where patient has >= 1 admission): ", hf['hadm_id'].unique().size)
print("Total # of HF admissions where patient has >= 2 admissions): ", hf_multiple_adm['hadm_id'].unique().size)

Total # of HF admissions where patient has >= 1 admission):  53024
Total # of HF admissions where patient has >= 2 admissions):  40728


## Combined cohort (OSA and HF)

In [124]:
# OSA and HF (1 or more admissions)
cohort_osa_hf = osa.merge(hf, on =['hadm_id', 'subject_id'], how ='outer')\
    .fillna(value = False)
cohort_osa_hf['group'] = cohort_osa_hf.apply(assign_group, axis=1)
cohort_osa_hf['group_name'] = cohort_osa_hf['group'].apply(group_str)

In [125]:
# OSA and HF (2 or more admissions)
cohort_osa_hf_multiple_adm = osa_multiple_adm.merge(hf_multiple_adm, on =['hadm_id', 'subject_id'], how ='outer')\
    .fillna(value = False)
cohort_osa_hf_multiple_adm['group'] = cohort_osa_hf_multiple_adm.apply(assign_group, axis=1)
cohort_osa_hf_multiple_adm['group_name'] = cohort_osa_hf_multiple_adm['group'].apply(group_str)

In [126]:
print("Cohort size for >= 1 admission (OSA/HF): ", cohort_osa_hf.shape[0])
print("Cohort size for >= 2 admission (OSA/HF): ", cohort_osa_hf_multiple_adm.shape[0])

Cohort size for >= 1 admission (OSA/HF):  77493
Cohort size for >= 2 admission (OSA/HF):  58218


In [127]:
# cohort_osa_hf.to_csv('Cohort/cohort_osa_hf.csv', index=False)
# cohort_osa_hf_multiple_adm.to_csv('Cohort/cohort_osa_hf_multiple_adm.csv', index=False)

## ADD NOTES COLUMN

### >= 1 admission   

In [128]:
cohort_osa_hf_notes = cohort_osa_hf.merge(notes, on = ['hadm_id', 'subject_id'] , how ='inner')
cohort_osa_hf_notes = cohort_osa_hf_notes[['subject_id', 'hadm_id', 'osa', 'hf', 'group', 'group_name', 'text']]

### >= 2 admissions

In [129]:
cohort_osa_hf_multiple_adm_notes = cohort_osa_hf_multiple_adm.merge(notes, on = ['hadm_id', 'subject_id'] , how ='inner')
cohort_osa_hf_multiple_adm_notes = cohort_osa_hf_multiple_adm_notes[['subject_id', 'hadm_id', 'osa', 'hf', 'group', 'group_name', 'text']]

In [130]:
cohort_osa_hf_notes.value_counts('group_name')

group_name
HF Only     41516
OSA Only    21929
OSA & HF     7963
Name: count, dtype: int64

In [131]:
cohort_osa_hf_notes.shape

(71408, 7)

<b>To CSV File</b>

In [132]:
# cohort_osa_hf_notes.to_csv('Cohort/cohort_osa_hf_notes.csv', index=False)
# cohort_osa_hf_multiple_adm_notes.to_csv('Cohort/cohort_osa_hf_multiple_adm_notes.csv', index=False)

# II. ADD DEMOGRAPHIC FEATURES
(mimiciv_hosp/admissions) <b>race, hospital expiration flag, insurance, length of stay, admission type</b>

(mimiciv_hosp/omr) <b>BMI</b>

(mimiciv_hosp/patients) <b>age, gender</b>

(mimiciv_derived/charlson) <b>Charlson comorbidity index</b>

In [133]:
# If not read in already, uncomment

# cohort_osa_hf_notes = pd.read_csv('cohort/cohort_osa_hf_notes.csv').reset_index(drop=True)
# cohort_osa_hf_multiple_adm_notes = pd.read_csv('cohort/cohort_osa_hf_multiple_adm_notes.csv').reset_index(drop=True)

# admissions = pd.read_csv(mimic4_path + 'hosp/admissions.csv')
# omr = pdf.read_csv(mimic4_path + 'hosp/omr.csv')
# patients = pd.read_csv(mimic4_path + 'hosp/patients.csv')
# charlson = pd.read_csv('/global/cfs/projectdirs/m1532/Projects_MVP/Sophia/derived/charlson.csv')

In [134]:
print(cohort_osa_hf_notes.shape[0])
cohort_osa_hf_notes.head(1)

71408


,subject_id,hadm_id,osa,hf,group,group_name,text
0,13340824,26708815,True,False,0,OSA Only,\nName: ___ Unit No: __...


## Merge cohort with admissions table
race, hospital expiration flag, insurance, length of stay, admission type

In [135]:
# Merge cohort with admissions table
cohort_osa_hf_notes_admissions = cohort_osa_hf_notes.merge(admissions,
                                                           on=['subject_id', 'hadm_id'],
                                                           how='inner').drop(['deathtime', 'admit_provider_id', \
                                                                              'admission_location', 'discharge_location', 'language', \
                                                                              'marital_status', 'edregtime', 'edouttime'], axis=1)

In [136]:
print("# of admissions: ", cohort_osa_hf_notes_admissions.shape[0])
cohort_osa_hf_notes_admissions.head(1)

# of admissions:  71408


,subject_id,hadm_id,osa,hf,group,group_name,text,admittime,dischtime,admission_type,insurance,race,hospital_expire_flag
0,13340824,26708815,True,False,0,OSA Only,\nName: ___ Unit No: __...,2162-09-01 21:40:00,2162-09-07 17:15:00,URGENT,Medicare,WHITE,0


#### race classification

In [137]:
cohort_osa_hf_notes_admissions['race_classified'] = cohort_osa_hf_notes_admissions['race'].apply(classify)

#### calculate length of stay

In [138]:
# https://github.com/aidotse/tabular-data-generation/blob/main/preprocessing/LOS-real-dataset-preparation.ipynb

# Convert admission and discharge times to datetime
cohort_osa_hf_notes_admissions['admittime'] = pd.to_datetime(cohort_osa_hf_notes_admissions['admittime'])
cohort_osa_hf_notes_admissions['dischtime'] = pd.to_datetime(cohort_osa_hf_notes_admissions['dischtime'])

# Calculate length of stay using admittime and dischtime
cohort_osa_hf_notes_admissions['los'] = (cohort_osa_hf_notes_admissions['dischtime'] - cohort_osa_hf_notes_admissions['admittime']).dt.total_seconds()/86400

In [139]:
# Remove extreme outliers (e.g., negative length of stay and value > 300)
def outlier_quatile(df, col_name):
    IQR = df[col_name].quantile(0.75) - df[col_name].quantile(0.25)
    higher_bound = df[col_name].quantile(0.75) + (IQR * 3)
    lower_bound = df[col_name].quantile(0.25) - (IQR * 3)
    return higher_bound, lower_bound

In [140]:
los_high, los_low = outlier_quatile(cohort_osa_hf_notes_admissions, 'los')
print(los_high, los_low)

print("Cohort size: ", cohort_osa_hf_notes_admissions.shape[0])
print("# of outlier LoS values: ", cohort_osa_hf_notes_admissions[((cohort_osa_hf_notes_admissions['los'] < 0) | (cohort_osa_hf_notes_admissions['los'] > los_high))].shape[0])

23.880381944444444 -13.731250000000003
Cohort size:  71408
# of outlier LoS values:  2034


In [141]:
cohort_osa_hf_notes_admissions = cohort_osa_hf_notes_admissions[((cohort_osa_hf_notes_admissions['los'] > 0) &\
                                                                 (cohort_osa_hf_notes_admissions['los'] < los_high))]

In [142]:
print(cohort_osa_hf_notes_admissions.shape[0])

69374


## Merge cohort with patients table
age, gender

In [143]:
cohort_osa_hf_notes_admissions_patients = cohort_osa_hf_notes_admissions.merge(patients, on='subject_id', how='inner').drop(['anchor_year_group', 'dod', 'race'], axis=1)

In [144]:
print("Number of admissions: ", cohort_osa_hf_notes_admissions_patients.shape[0])
cohort_osa_hf_notes_admissions_patients.head(1)

Number of admissions:  69374


,subject_id,hadm_id,osa,hf,group,group_name,text,admittime,dischtime,admission_type,insurance,hospital_expire_flag,race_classified,los,gender,anchor_age,anchor_year
0,13340824,26708815,True,False,0,OSA Only,\nName: ___ Unit No: __...,2162-09-01 21:40:00,2162-09-07 17:15:00,URGENT,Medicare,0,W,5.815972,F,91,2162


#### calculate age

In [145]:
# Convert admission year to datetime
cohort_osa_hf_notes_admissions_patients['admittime_year'] = pd.DatetimeIndex(cohort_osa_hf_notes_admissions_patients['admittime']).year

In [146]:
# the age of a patient = hospital admission time - anchor_year + anchor_age
cohort_osa_hf_notes_admissions_patients['age'] = cohort_osa_hf_notes_admissions_patients['admittime_year'] - cohort_osa_hf_notes_admissions_patients['anchor_year'] + cohort_osa_hf_notes_admissions_patients['anchor_age']

## Merge cohort with charlson table
Charlson comorbidity index

In [147]:
charlson_filtered = charlson[['subject_id', 'hadm_id', 'charlson_comorbidity_index']]

In [148]:
cohort_osa_hf_notes_admissions_patients_charlson = cohort_osa_hf_notes_admissions_patients.merge(charlson_filtered, on=['subject_id', 'hadm_id'], how='inner')\
    .drop(['anchor_age', 'anchor_year', 'admittime_year', 'admittime', 'dischtime'], axis=1)\
    .rename(columns={'charlson_comorbidity_index':'charlson'})

In [149]:
print("Number of admissions: ", cohort_osa_hf_notes_admissions_patients_charlson.shape[0])
cohort_osa_hf_notes_admissions_patients_charlson.head(1)

Number of admissions:  69374


,subject_id,hadm_id,osa,hf,group,group_name,text,admission_type,insurance,hospital_expire_flag,race_classified,los,gender,age,charlson
0,13340824,26708815,True,False,0,OSA Only,\nName: ___ Unit No: __...,URGENT,Medicare,0,W,5.815972,F,91,4


## Merge cohort with OMR table
BMI (patient level)

In [150]:
add_bmi = cohort_osa_hf_notes_admissions_patients_charlson\
    .merge(omr[['subject_id', 'result_name', 'result_value']], on='subject_id', how='inner')

In [151]:
# Extract rows that have BMI recorded
add_bmi = add_bmi[add_bmi['result_name'].str.contains("BMI")]
add_bmi['result_value'] = add_bmi['result_value'].astype(float)
add_bmi = add_bmi.rename(columns={'result_value': 'bmi'})\
    .drop('result_name', axis='columns')

In [152]:
# Since BMI is on patient level, get median value from all admissions per patient
cohort_osa_hf_notes_admissions_patients_charlson_bmi = add_bmi\
    .groupby(['subject_id', 'hadm_id'])['bmi'].median().reset_index()\
    .merge(add_bmi.drop('bmi', axis='columns'), on=['subject_id', 'hadm_id'])\
    .drop_duplicates(subset=['subject_id', 'hadm_id'])

In [153]:
cohort_osa_hf_notes_admissions_patients_charlson_bmi.head(3)

,subject_id,hadm_id,bmi,osa,hf,group,group_name,text,admission_type,insurance,hospital_expire_flag,race_classified,los,gender,age,charlson
0,10000980,20897796,31.6,False,True,2,HF Only,\nName: ___ Unit No: ___\n \nAdmi...,OBSERVATION ADMIT,Other,0,B,2.587500,F,80,9
73,10000980,24947999,31.6,False,True,2,HF Only,\nName: ___ Unit No: ___\n \nAdmi...,EW EMER.,Medicare,0,B,1.792361,F,77,9
146,10000980,25242409,31.6,False,True,2,HF Only,\nName: ___ Unit No: ___\n \nAdmi...,EW EMER.,Medicare,0,B,7.897917,F,78,10


In [154]:
print("Cohort size after adding BMI data: ", cohort_osa_hf_notes_admissions_patients_charlson_bmi.shape[0])

Cohort size after adding BMI data:  54220


<b>Remove outliers</b>

In [155]:
print("# of abnormal BMI values: ", \
      cohort_osa_hf_notes_admissions_patients_charlson_bmi\
      [((cohort_osa_hf_notes_admissions_patients_charlson_bmi['bmi'] < 1) |\
        (cohort_osa_hf_notes_admissions_patients_charlson_bmi['bmi'] > 250))]\
      .shape[0])

# of abnormal BMI values:  16


In [156]:
cohort_osa_hf_notes_admissions_patients_charlson_bmi.value_counts('bmi').sort_index()

bmi
0.40       1
0.80       2
2.15       2
2.45       1
4.70       2
          ..
2250.00    2
3093.20    1
4554.70    1
4881.90    3
6102.00    3
Name: count, Length: 1265, dtype: int64

In [157]:
cohort_osa_hf_notes_admissions_patients_charlson_bmi_cleaned = cohort_osa_hf_notes_admissions_patients_charlson_bmi\
    [((cohort_osa_hf_notes_admissions_patients_charlson_bmi['bmi'] > 1) &\
      (cohort_osa_hf_notes_admissions_patients_charlson_bmi['bmi'] < 250))]

In [158]:
print("Cohort size after removing outlier BMI values: ", cohort_osa_hf_notes_admissions_patients_charlson_bmi_cleaned.shape[0])

Cohort size after removing outlier BMI values:  54204


In [159]:
cohort_osa_hf_notes_admissions_patients_charlson_bmi_cleaned.to_csv(root_path + 'Cohort/cohort_osa_hf_notes_admissions_patients_charlson_bmi.csv', index=False)

# III. ADD LABEVENTS

## Merge labevents tables

In [160]:
osa_hf_m1 = pd.read_csv(root_path + 'Cohort/cohort_osa_hf_notes_admissions_patients_charlson_bmi.csv')

In [161]:
osa_hf_m1

,subject_id,hadm_id,bmi,osa,hf,group,group_name,text,admission_type,insurance,hospital_expire_flag,race_classified,los,gender,age,charlson
0,10000980,20897796,31.60,False,True,2,HF Only,\nName: ___ Unit No: ___\n \nAdmi...,OBSERVATION ADMIT,Other,0,B,2.587500,F,80,9
1,10000980,24947999,31.60,False,True,2,HF Only,\nName: ___ Unit No: ___\n \nAdmi...,EW EMER.,Medicare,0,B,1.792361,F,77,9
2,10000980,25242409,31.60,False,True,2,HF Only,\nName: ___ Unit No: ___\n \nAdmi...,EW EMER.,Medicare,0,B,7.897917,F,78,10
3,10000980,26913865,31.60,False,True,2,HF Only,\nName: ___ Unit No: ___\n \nAdmi...,EW EMER.,Medicare,0,B,5.806944,F,76,8
4,10000980,29654838,31.60,False,True,2,HF Only,\nName: ___ Unit No: ___\n \nAdmi...,EW EMER.,Medicare,0,B,1.992361,F,75,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54199,19998330,24096838,25.00,True,True,1,OSA & HF,\nName: ___ Unit No: ...,EW EMER.,Other,0,B,3.804861,F,72,8
54200,19998330,24492004,25.00,True,True,1,OSA & HF,\nName: ___ Unit No: ...,EW EMER.,Other,0,B,7.425000,F,72,8
54201,19998330,27282608,25.00,True,True,1,OSA & HF,\nName: ___ Unit No: ...,EW EMER.,Medicare,0,B,1.397917,F,71,8
54202,19998350,21130518,41.65,True,False,0,OSA Only,\nName: ___ Unit No: ___\n...,EU OBSERVATION,Other,0,B,0.875694,M,56,4


In [162]:
d_labitems = pd.read_csv(mimic4_path + 'hosp/d_labitems.csv')
print(d_labitems.shape)
d_labitems.columns

(1622, 4)


Index(['itemid', 'label', 'fluid', 'category'], dtype='object')

In [ ]:
labevents = pd.read_csv(mimic4_path + 'hosp/labevents.csv')

In [ ]:
print(labevents.shape)
labevents.columns

In [ ]:
#using cleaned cohort (without patients that have negative/outlier LOS)
osa_hf_labevents = osa_hf_m1.merge(labevents, how = 'inner', on = ['subject_id', 'hadm_id'])
osa_hf_labevents = osa_hf_labevents.merge(d_labitems, how = 'inner', on = 'itemid')

In [ ]:
labels_labevents = osa_hf_labevents[['itemid', 'label']].drop_duplicates(subset='itemid')
labels_labevents = pd.Series(labels_labevents['label'].values,index=labels_labevents['itemid'].apply(str).values)

In [ ]:
labels_labevents.shape[0]

In [ ]:
# labels_labevents.to_csv(root_path + 'Data/labels_labevents.csv')

## Keep labevents that are recorded in more than a certain % of admissions

In [ ]:
labevents_unique = osa_hf_labevents['itemid'].unique()
labevents_unique.size

In [ ]:
print("# of admissions in original, cleaned cohort: ", osa_hf_m1['hadm_id'].unique().size)
print("# of admissions after merging with labevents: ", osa_hf_labevents['hadm_id'].unique().size)

### calculate percentage of patients for each lab

In [ ]:
unique_itemid = osa_hf_labevents['itemid'].unique()
print("# of unique itemid: ", unique_itemid.size) # Note: Some itemids share lab name with another itemid

In [ ]:
# For each lab, calculate percentage of patients that have the lab recorded
itemid_percentage_list = []
for itemid in tqdm(unique_itemid):
    itemid_rows = osa_hf_labevents[osa_hf_labevents['itemid'] == itemid]
    percentage = itemid_rows['hadm_id'].unique().size / osa_hf_labevents['hadm_id'].unique().size * 100
    itemid_percentage_list.append([itemid, percentage])

In [ ]:
# Convert list to DataFrame
itemid_percentage_df = pd.DataFrame(itemid_percentage_list, columns=["itemid", "percentage"])
itemid_percentage_df.sort_values(by=["percentage"], ascending=False, ignore_index=True, inplace=True)
print("Lowest percentages: ")
itemid_percentage_df.tail(10)

<b>To CSV file</b>

In [ ]:
# itemid_percentage_df.to_csv(root_path + 'Data/common_labevents.csv')

In [ ]:
n = 0.5 # customized percentage
value = itemid_percentage_df[itemid_percentage_df["percentage"] > n].shape[0]
print(f"Number of itemid in > {n} % of admissions: {value}")

In [ ]:
# If doing cutoff percentage value
itemid_keep = itemid_percentage_df[itemid_percentage_df["percentage"] > n]
mask = osa_hf_labevents['itemid'].isin(list(itemid_keep['itemid']))
osa_hf_labevents_cleaned = osa_hf_labevents[mask]

In [ ]:
osa_hf_labevents_cleaned['itemid'].unique().size

In [ ]:
osa_hf_labevents_cleaned.columns

In [ ]:
print(osa_hf_m1.shape[0])
print(osa_hf_labevents_cleaned['hadm_id'].unique().size)
osa_hf_labevents_cleaned.shape[0]

## Processing
for each lab (admission-level), get count of abnormal value recordings

<i>Note: Some hadm_id disappear due to NaN value in ref_range_lower and/or ref_range_upper.</i>

In [ ]:
lab_values = osa_hf_labevents_cleaned.groupby(['subject_id', 'hadm_id','itemid','ref_range_lower','ref_range_upper'])['valuenum'].median()
lab_values = lab_values.reset_index(name='valuenum')

In [ ]:
print(lab_values['hadm_id'].unique().size)
lab_values.head(10)

In [ ]:
# Trace why some hadm_id disappear
# Some hadm_id disappear due to NaN value in ref_range_lower and/or ref_range_upper.
test = osa_hf_labevents_cleaned[~osa_hf_labevents_cleaned['hadm_id'].isin(list(lab_values['hadm_id']))]
print(test['hadm_id'].unique().size)
test.head(3)

### without flag

In [ ]:
lab_values = osa_hf_labevents_cleaned.groupby(['subject_id', 'hadm_id','itemid','ref_range_lower','ref_range_upper'])['valuenum'].median()
lab_values = lab_values.reset_index(name='valuenum')

In [ ]:
print(lab_values['hadm_id'].unique().size)
lab_values.head(10)

In [ ]:
# Trace why some hadm_id disappear
# Some hadm_id disappear due to NaN value in ref_range_lower and/or ref_range_upper.
test = osa_hf_labevents_cleaned[~osa_hf_labevents_cleaned['hadm_id'].isin(list(lab_values['hadm_id']))]
print(test['hadm_id'].unique().size)
test.head(3)

In [ ]:
# Get rows where lab values are outside normal range
osa_hf_labevents_cleaned_abnormal_values = lab_values[(lab_values['valuenum'] < lab_values['ref_range_lower']) \
                                                      | (lab_values['valuenum'] > lab_values['ref_range_upper'])].drop(['ref_range_lower', 'ref_range_upper'], axis='columns')
osa_hf_labevents_cleaned_abnormal_values.head(5)

In [ ]:
osa_hf_labevents_cleaned_abnormal_values_count = osa_hf_labevents_cleaned_abnormal_values.groupby(['subject_id', 'hadm_id','itemid'])['valuenum'].count()
osa_hf_labevents_cleaned_abnormal_values_count_df = osa_hf_labevents_cleaned_abnormal_values_count.reset_index(name='count')

print(osa_hf_labevents_cleaned_abnormal_values_count_df.shape[0])
osa_hf_labevents_cleaned_abnormal_values_count_df.head(3)

In [ ]:
osa_hf_labevents_cleaned_abnormal_values_count_df[osa_hf_labevents_cleaned_abnormal_values_count_df['count'] >= 2].shape

In [ ]:
# Pivot DataFrame -> itemid as columns
df_pivot = osa_hf_labevents_cleaned_abnormal_values_count_df\
    .pivot(index=['subject_id', 'hadm_id'], columns=['itemid'], values='count')

# Keep all rows, regardless if they have an abnormal value recorded
osa_hf_labevents_cleaned_abnormal_values_count_df_pivot = df_pivot\
    .rename_axis(columns=None)\
    .reset_index()

osa_hf_labevents_cleaned_abnormal_values_count_df_pivot = osa_hf_labevents_cleaned_abnormal_values_count_df_pivot\
    .merge(lab_values[['subject_id', 'hadm_id']].drop_duplicates(subset=['subject_id', 'hadm_id']), on=['subject_id', 'hadm_id'], how='outer', validate="1:1")\
    .fillna(0)\
    .astype(int)

print(osa_hf_labevents_cleaned_abnormal_values_count_df_pivot.shape)
osa_hf_labevents_cleaned_abnormal_values_count_df_pivot.head(3)

### with flag

In [ ]:
lab_values_flag = osa_hf_labevents_cleaned[['subject_id', 'hadm_id','itemid','flag']]

In [ ]:
# Get rows where lab values are outside normal range
osa_hf_labevents_cleaned_abnormal_flag = lab_values_flag[lab_values_flag['flag'] == 'abnormal']
osa_hf_labevents_cleaned_abnormal_flag.head(5)

In [ ]:
osa_hf_labevents_cleaned_abnormal_flag_count = osa_hf_labevents_cleaned_abnormal_flag.groupby(['subject_id', 'hadm_id','itemid'])['flag'].count()
osa_hf_labevents_cleaned_abnormal_flag_count_df = osa_hf_labevents_cleaned_abnormal_flag_count.reset_index(name='count')

print(osa_hf_labevents_cleaned_abnormal_flag_count_df.shape[0])
osa_hf_labevents_cleaned_abnormal_flag_count_df.head(3)

In [ ]:
osa_hf_labevents_cleaned_abnormal_flag_count_df.value_counts('count')

In [ ]:
# Pivot DataFrame -> itemid as columns ?? 3rd section review later
df_pivot = osa_hf_labevents_cleaned_abnormal_flag_count_df\
    .pivot(index=['subject_id', 'hadm_id'], columns=['itemid'], values='count')

# Keep all rows, regardless if they have an abnormal value recorded
osa_hf_labevents_cleaned_abnormal_flag_count_df_pivot = df_pivot\
    .rename_axis(columns=None)\
    .reset_index()

osa_hf_labevents_cleaned_abnormal_flag_count_df_pivot = osa_hf_labevents_cleaned_abnormal_flag_count_df_pivot\
    .merge(lab_values[['subject_id', 'hadm_id']].drop_duplicates(subset=['subject_id', 'hadm_id']), on=['subject_id', 'hadm_id'], how='outer', validate="1:1")\
    .fillna(0)\
    .astype(int)

print(osa_hf_labevents_cleaned_abnormal_flag_count_df_pivot.shape)
osa_hf_labevents_cleaned_abnormal_flag_count_df_pivot.head(3)

#### merge with original cohort

In [ ]:
print(osa_hf_labevents_cleaned.columns)

In [ ]:
osa_hf_labevents_abnormal_count = osa_hf_labevents_cleaned.loc[:, 'subject_id':'charlson']\
    .drop_duplicates(subset=['subject_id', 'hadm_id'])\
    .merge(osa_hf_labevents_cleaned_abnormal_flag_count_df_pivot, on=['subject_id', 'hadm_id'], how="inner")

In [ ]:
print(osa_hf_labevents_abnormal_count.shape)

<b>To CSV file</b>

In [ ]:
osa_hf_labevents_abnormal_count.to_csv(root_path + 'Cohort/osa_hf_labevents_abnormal_count_2.csv', index=False)

# IV. ADD DIAGNOSIS

## Merge diagnosis table

In [ ]:
osa_hf_m1 = pd.read_csv(root_path + 'Cohort/cohort_osa_hf_notes_admissions_patients_charlson_bmi.csv')

In [ ]:
print(osa_hf_m1.shape)
print(osa_hf_m1.columns)

In [ ]:
d_icd_diagnoses = pd.read_csv(mimic4_path + 'hosp/d_icd_diagnoses.csv')
print(d_icd_diagnoses.shape)
d_icd_diagnoses.columns

In [ ]:
diagnoses = pd.read_csv(mimic4_path + 'hosp/diagnoses_icd.csv')

In [ ]:
print(diagnoses.shape)
diagnoses.columns

In [ ]:
#using cleaned cohort (without patients that have negative/outlier LOS)
osa_hf_diagnoses_1 = osa_hf_m1.merge(diagnoses, how = 'inner', on = ['subject_id', 'hadm_id'])
osa_hf_diagnoses = osa_hf_diagnoses_1.merge(d_icd_diagnoses, how = 'inner', on = ['icd_code', 'icd_version'])

In [ ]:
osa_codes = pd.read_csv(root_path + 'icd_codes/osa.csv')
hf_codes = pd.read_csv(root_path + 'icd_codes/heart_failure.csv')

## Clean diagnosis file
Remove special characters

In [ ]:
osa_code_list = list(osa_codes['icd_code'].astype(str))
osa_code_list = [re.sub(r'\xa0$', '', code) for code in osa_code_list]

In [ ]:
len(list(osa_codes['icd_code']))

In [ ]:
hf_code_list = list(hf_codes['icd_code'].astype(str))
hf_code_list = [re.sub(r'\xa0$', '', code) for code in hf_code_list]

In [ ]:
len(hf_code_list)

## Remove rows with OSA and HF ICD diagnosis codes

In [ ]:
# remove diagnoses for OSA and HF
mask = (osa_hf_diagnoses['icd_code'].isin(osa_code_list)) | (osa_hf_diagnoses['icd_code'].isin(hf_code_list))
diagnoses_updated = osa_hf_diagnoses[~mask]

In [ ]:
print("Original cohort shape: ", osa_hf_m1.shape)
print("Diagnoses before filtering shape: ", osa_hf_diagnoses.shape)
print("Filtered diagnoses shape: ", diagnoses_updated.shape)

In [ ]:
print("# of ICD codes before removing OSA and HF codes: ", osa_hf_diagnoses['icd_code'].unique().size)
print("# of ICD Codes after removing OSA and HF codes: ", diagnoses_updated['icd_code'].unique().size)

## Keep ICD codes that are recorded in more than a certain % of admissions

In [ ]:
icd_code_osa_only = diagnoses_updated[diagnoses_updated['group'] == 0]
icd_code_hf_only = diagnoses_updated[diagnoses_updated['group'] == 2]
icd_code_osa_hf = diagnoses_updated[diagnoses_updated['group'] == 1]

In [ ]:
unique_icd_code_osa = icd_code_osa_only['icd_code'].unique()
unique_icd_code_hf = icd_code_hf_only['icd_code'].unique()
unique_icd_code_osa_hf = icd_code_osa_hf['icd_code'].unique()

print("# of unique ICD codes in OSA only cohort: ", unique_icd_code_osa.size)
print("# of unique ICD codes in HF only cohort: ", unique_icd_code_hf.size)
print("# of unique ICD codes in OSA + HF cohort: ", unique_icd_code_osa_hf.size)
# test = unique_icd_code_osa[0:10]

In [ ]:
unique_icd_codes = diagnoses_updated['icd_code'].unique()

### For each ICD code, calculate percentage of admissions that have the ICD code recorded
Note: takes a while to run.

#### most common ICD codes for each group

In [ ]:
icd_code_percentage_list_osa = []
for icd_code in tqdm(unique_icd_code_osa):
    icd_code_rows = icd_code_osa_only[icd_code_osa_only['icd_code'] == icd_code]
    percentage = icd_code_rows['hadm_id'].unique().size / icd_code_osa_only['hadm_id'].unique().size * 100
    if percentage > 0.001:
        icd_code_percentage_list_osa.append([icd_code, percentage])

In [ ]:
icd_code_percentage_list_hf = []
for icd_code in tqdm(unique_icd_code_hf):
    icd_code_rows = icd_code_hf_only[icd_code_hf_only['icd_code'] == icd_code]
    percentage = icd_code_rows['hadm_id'].unique().size / icd_code_hf_only['hadm_id'].unique().size * 100
    icd_code_percentage_list_hf.append([icd_code, percentage])

In [ ]:
icd_code_percentage_list_osa_hf = []
for icd_code in tqdm(unique_icd_code_osa_hf):
    icd_code_rows = icd_code_osa_hf[icd_code_osa_hf['icd_code'] == icd_code]
    percentage = icd_code_rows['hadm_id'].unique().size / icd_code_osa_hf['hadm_id'].unique().size * 100
    icd_code_percentage_list_osa_hf.append([icd_code, percentage])

In [ ]:
# Convert list to DataFrame
osa_common_icd_codes = pd.DataFrame(icd_code_percentage_list_osa, columns=["icd_code", "percentage"]).sort_values(by=["percentage"], ascending=False, ignore_index=True)
hf_common_icd_codes = pd.DataFrame(icd_code_percentage_list_hf, columns=["icd_code", "percentage"]).sort_values(by=["percentage"], ascending=False, ignore_index=True)
osa_hf_common_icd_codes = pd.DataFrame(icd_code_percentage_list_osa_hf, columns=["icd_code", "percentage"]).sort_values(by=["percentage"], ascending=False, ignore_index=True)

In [ ]:
osa_common_icd_codes.to_csv(root_path + 'Data/osa_common_icd_codes.csv', index=False)
hf_common_icd_codes.to_csv(root_path + 'Data/hf_common_icd_codes.csv', index=False)
osa_hf_common_icd_codes.to_csv(root_path + 'Data/osa_hf_common_icd_codes.csv', index=False)

#### most common ICD codes for all groups

In [ ]:
# For each ICD code, calculate percentage of patients that have the ICD code recorded
icd_code_percentage_list = []
for icd_code in tqdm(unique_icd_codes):
    icd_code_rows = diagnoses_updated[diagnoses_updated['icd_code'] == icd_code]
    percentage = icd_code_rows['hadm_id'].unique().size / diagnoses_updated['hadm_id'].unique().size * 100
    icd_code_percentage_list.append([icd_code, percentage])

In [ ]:
# Convert list to DataFrame
icd_code_percentage_df = pd.DataFrame(icd_code_percentage_list, columns=["icd_code", "percentage"])
icd_code_percentage_df_label = icd_code_percentage_df.merge(d_icd_diagnoses, on='icd_code')[['icd_code', 'long_title', 'percentage']]\
    .sort_values(by=["percentage"], ascending=False, ignore_index=True)
print("Lowest percentages: ")
icd_code_percentage_df_label.tail(10)

In [ ]:
# icd_code_percentage_df_label.to_csv(root_path + 'Data/common_icd_codes.csv', index=False)

## Summary

In [ ]:
icd_code_percentage_df_label = pd.read_csv(root_path + 'Data/common_icd_codes.csv')
osa_common_icd_codes = pd.read_csv(root_path + 'Data/osa_common_icd_codes.csv')
hf_common_icd_codes = pd.read_csv(root_path + 'Data/hf_common_icd_codes.csv')
osa_hf_common_icd_codes = pd.read_csv(root_path + 'Data/osa_hf_common_icd_codes.csv')

In [ ]:
n = 0.5 # customized percentage
value = icd_code_percentage_df_label[icd_code_percentage_df_label["percentage"] > n].shape[0]
osa_icd_code_count = osa_common_icd_codes[osa_common_icd_codes["percentage"] > n].shape[0]
hf_icd_code_count = hf_common_icd_codes[hf_common_icd_codes["percentage"] > n].shape[0]
osa_hf_icd_code_count = osa_hf_common_icd_codes[osa_hf_common_icd_codes["percentage"] > n].shape[0]

print("Total count of ICD codes: ", icd_code_percentage_df_label.shape[0])
print(f"Count of ICD codes in > {n} % of all admissions: {value}\n")
# print(f"Count of ICD codes in > {n} % of OSA admissions: {osa_icd_code_count}")
# print(f"Count of ICD codes in > {n} % of HF admissions: {hf_icd_code_count}")
# print(f"Count of ICD codes in > {n} % of OSA & HF admissions: {osa_hf_icd_code_count}")

In [ ]:
icd_code_percentage_df_label['icd_code'].unique().size

In [ ]:
diagnoses_updated['hadm_id'].unique().size

In [ ]:
# If doing cutoff percentage value
icd_code_keep = icd_code_percentage_df_label[icd_code_percentage_df_label["percentage"] > n]
mask = diagnoses_updated['icd_code'].isin(list(icd_code_keep['icd_code']))
osa_hf_diagnoses_filtered = diagnoses_updated[mask]

### remove ICD codes that are duplicated
Note: Some ICD codes are duplicated because they are the same in version 9 and 10.

#### trace why there are duplicate values after cutoff

In [ ]:
icd_code_percentage_df_label[icd_code_percentage_df_label["percentage"] > n]['icd_code'].shape[0]

In [ ]:
icd_code_percentage_df_label[icd_code_percentage_df_label["percentage"] > n]['icd_code'].unique().size

In [ ]:
no_dup = icd_code_percentage_df_label[icd_code_percentage_df_label["percentage"] > n]\
    .drop_duplicates(subset=['icd_code'])
print(no_dup.shape[0])
print(icd_code_keep.shape[0])

In [ ]:
dup_codes = icd_code_keep[icd_code_keep['icd_code'].isin(list(no_dup['icd_code']))]
mask = icd_code_keep.duplicated(subset=['icd_code'])
dup_codes = icd_code_keep[mask]

In [ ]:
dup_codes

In [ ]:
icd_code_keep[icd_code_keep['icd_code'] == 'V462']

#### remove duplicate codes

In [ ]:
# Filter out duplicate codes
mask = osa_hf_diagnoses_filtered['icd_code'].isin(list(dup_codes['icd_code']))
osa_hf_diagnoses_filtered = osa_hf_diagnoses_filtered[~mask]

In [ ]:
osa_hf_diagnoses_filtered['icd_code'].unique().size

## Explore hypoxia diagnoses

In [ ]:
hypoxia_icd_code_list = ['R0902', 'J9600', 'J9601', 'J9610', 'J9611', 'J9620', 'J9621', '79902', '51881', '51883', '51884', '79901', 'J9612', 'J9622', 'Z9981']

In [ ]:
hypoxia_patient_percentage = icd_code_percentage_df_label[icd_code_percentage_df_label['icd_code'].isin(hypoxia_icd_code_list)]\
    .sort_values('percentage', ascending=False).reset_index(drop=True)

In [ ]:
hypoxia_patient_percentage

In [ ]:
mask = diagnoses_updated['icd_code'].isin(hypoxia_icd_code_list)
osa_hf_diagnoses_hypoxia = diagnoses_updated[mask]

In [ ]:
print("# of unique admissions before filtering: ", diagnoses_updated['hadm_id'].unique().size)
print("# of unique admissions after filtering: ", osa_hf_diagnoses_filtered['hadm_id'].unique().size, "\n")
print("# of unique admissions with hypoxia ICD diagnosis code: ", osa_hf_diagnoses_hypoxia['hadm_id'].unique().size)

In [ ]:
osa_hf_diagnoses_filtered['icd_code'].unique().size

<b>Add hypoxia ICD codes regardless of percentage</b>

In [ ]:
osa_hf_diagnoses_final = osa_hf_diagnoses_filtered.merge(osa_hf_diagnoses_hypoxia, how='outer')

In [ ]:
osa_hf_diagnoses_final['icd_code'].unique().size

In [ ]:
osa_hf_diagnoses_final.shape[0]

In [ ]:
osa_hf_diagnoses_final.columns

In [ ]:
osa_hf_diagnoses_final.head(5)

In [ ]:
print("# of unique hadm_id in original cohort: ", osa_hf_diagnoses['hadm_id'].unique().size)
print("# of unique hadm_id after adding diagnoses: ", osa_hf_diagnoses_final['hadm_id'].unique().size)
print("# of unique ICD diagnosis codes: ", osa_hf_diagnoses_final['icd_code'].unique().size)

## Processing
for each ICD code (patient-level), get # of diagnosis count

In [ ]:
icd_code_count = osa_hf_diagnoses_final.groupby(['subject_id', 'hadm_id', 'icd_code', 'long_title'])['seq_num'].count()
icd_code_count_df = icd_code_count.reset_index(name='icd_code_count')
print(icd_code_count_df.shape[0])
icd_code_count_df.head(10)

In [ ]:
# Save long_title for ICD codes
labels_icd_code_filtered = icd_code_count_df.drop_duplicates(subset='icd_code')
labels_icd_code_filtered = pd.Series(labels_icd_code_filtered['long_title'].values,index=labels_icd_code_filtered['icd_code'].apply(str).values)

In [ ]:
labels_icd_code_filtered.to_csv(root_path + 'Data/labels_icd_code_filtered.csv')

In [ ]:
osa_hf_diagnoses_final_pivot = icd_code_count_df.pivot(index=['subject_id', 'hadm_id'], columns='icd_code', values='icd_code_count')
print(osa_hf_diagnoses_final_pivot.shape)
osa_hf_diagnoses_final_pivot.head(3)

In [ ]:
osa_hf_diagnoses_final_pivot_df = osa_hf_diagnoses_final_pivot\
    .rename_axis(columns=None)\
    .reset_index()\
    .fillna(0)

<b>Look if there are nonbinary values (there should only be 1 count for each diagnosis)</b>

In [ ]:
print(osa_hf_diagnoses_final_pivot_df[osa_hf_diagnoses_final_pivot_df.isin([2]).any(axis=1)].shape[0])
osa_hf_diagnoses_final_pivot_df.columns[osa_hf_diagnoses_final_pivot_df.isin([2]).any(axis=0)]

In [ ]:
# Replace nonbinary values
osa_hf_diagnoses_final_pivot_df.where(osa_hf_diagnoses_final_pivot_df != 2, 1, inplace=True)

## Merge with cohort

In [ ]:
osa_hf_diagnoses_final_pivot_df.shape

In [ ]:
print(osa_hf_diagnoses_final_pivot_df['hadm_id'].unique().size)
print(osa_hf_diagnoses_filtered['hadm_id'].unique().size)
print(osa_hf_diagnoses_filtered.columns)

In [ ]:
osa_hf_icd_code_count = osa_hf_diagnoses_filtered.loc[:, 'subject_id':'charlson']\
    .drop_duplicates(subset=['subject_id', 'hadm_id'])\
    .merge(osa_hf_diagnoses_final_pivot_df, on=['subject_id', 'hadm_id'], how="inner")

In [ ]:
print(osa_hf_icd_code_count.shape)
# osa_hf_labevents_count.head(5)

In [ ]:
# to CSV
# osa_hf_icd_code_count.to_csv(root_path + 'Cohort/osa_hf_diagnosis_count.csv', index=False)

### Read CSV

In [ ]:
data = pd.read_csv(root_path + "2025/cohort/osa_hf_all_features_labeled.csv")

In [ ]:
data_id_only = data.loc[:, ['subject_id', 'hadm_id']]

In [ ]:
emar = pd.read_csv(mimic4_path + 'hosp/emar.csv')

In [ ]:
emar_detail = pd.read_csv(mimic4_path + 'hosp/emar_detail.csv', low_memory=False)

In [ ]:
pharmacy = pd.read_csv(mimic4_path + 'hosp/pharmacy.csv')

In [ ]:
procedures_icd = pd.read_csv(mimic4_path + 'hosp/procedures_icd.csv')

In [ ]:
d_icd_procedures = pd.read_csv(mimic4_path + 'hosp/d_icd_procedures.csv')

In [ ]:
data.shape

In [ ]:
data.columns

# V. ADD MEDICATIONS

Use binary value to represent medications

In [ ]:
# merge medications (emar, emar_detail, and pharmacy)
osa_hf_med = data_id_only.merge(emar, how = 'left', on = ['subject_id', 'hadm_id'])
osa_hf_med = osa_hf_med.merge(emar_detail[['subject_id', 'emar_id', 'emar_seq', 'dose_given', 'dose_given_unit', 'pharmacy_id']], how = 'left', on = ['subject_id','emar_id', 'emar_seq', 'pharmacy_id'])

In [ ]:
osa_hf_med = osa_hf_med.merge(pharmacy[['subject_id', 'hadm_id', 'pharmacy_id', 'medication', 'starttime', 'stoptime']], how = 'left', on = ['subject_id','hadm_id', 'pharmacy_id', 'medication'])

In [ ]:
osa_hf_med.columns

In [ ]:
print("Unique patients: ", osa_hf_med['subject_id'].unique().size)
print("Unique admissions: ", osa_hf_med['hadm_id'].unique().size)

In [ ]:
medications = osa_hf_med[['pharmacy_id', 'medication']].drop_duplicates(subset='pharmacy_id')
# medications = pd.Series(medications['medication'].values,index=medications['emar_id'].apply(str).values)

In [ ]:
medications

In [ ]:
medications['medication'].unique().size

In [ ]:
medications['pharmacy_id'].unique().size

In [ ]:
osa_hf_med['medication'].unique().size

#### only keep certain percentage % of medications

In [ ]:
unique_med = osa_hf_med['medication'].unique()
print("# of unique medications: ", unique_med.size)

<b>For each lab, calculate percentage of patients that have the lab recorded</b>

Uncomment only if needed!

In [ ]:
## -- Takes a while

# med_percentage_list = []
# for med in tqdm(unique_med):
#     med_rows = osa_hf_med[osa_hf_med['medication'] == med]
#     percentage = med_rows['hadm_id'].unique().size / osa_hf_med['hadm_id'].unique().size * 100
#     med_percentage_list.append([med, percentage])

# # Convert list to DataFrame
# med_percentage_df = pd.DataFrame(med_percentage_list, columns=["medication", "percentage"])
# med_percentage_df.sort_values(by=["percentage"], ascending=False, ignore_index=True, inplace=True)
# print("Lowest percentages: ")
# med_percentage_df.tail(10)

<b>To CSV file (medications for OSA-HF cohort)</b>

In [ ]:
# -- Uncomment if needed --
# med_percentage_df.to_csv(root_path + '2025/data/common_medications.csv')

med_percentage_df = pd.read_csv(root_path + '2025/data/common_medications.csv')

In [ ]:
n = 0.5 # customized percentage
value = med_percentage_df[med_percentage_df["percentage"] > n].shape[0]
print(f"Number of medications in > {n} % of admissions: {value}")

In [ ]:
# If doing cutoff percentage value

med_keep = med_percentage_df[med_percentage_df["percentage"] > n]

osa_hf_med_cleaned = osa_hf_med.copy()

osa_hf_med_cleaned['medication'] = osa_hf_med['medication'].where(
    osa_hf_med['medication'].isin(med_keep['medication']), 
    other=pd.NA  # or use np.nan if preferred
)

## -- OLD CODE -- for removing admissions
# mask = osa_hf_med['medication'].isin(list(med_keep['medication']))
# osa_hf_med_cleaned = osa_hf_med[mask]

In [ ]:
osa_hf_med_cleaned['medication'].unique().size

In [ ]:
osa_hf_med_cleaned.columns

In [ ]:
print(data.shape[0])
print(osa_hf_med_cleaned['hadm_id'].unique().size)
osa_hf_med_cleaned.shape[0]

In [ ]:
# -- AFTER DROPPING DUPLICATES
osa_hf_med_cleaned.drop_duplicates(inplace=True)

print(data.shape[0])
print(osa_hf_med_cleaned['hadm_id'].unique().size)
osa_hf_med_cleaned.shape[0]

In [ ]:
# -- Many rows have null values in some columns (e.g. dose_given)
df_no_null = osa_hf_med_cleaned.dropna()
print(df_no_null['hadm_id'].nunique())
df_no_null.head(3)

In [ ]:
osa_hf_med_cleaned.head(2)

In [ ]:
osa_hf_med_cleaned_count = osa_hf_med_cleaned.groupby(['subject_id', 'hadm_id','medication'])['charttime'].count()

## -- Uncomment if using binary value (true or false if admission has particular medication) --
osa_hf_med_cleaned_count.where(osa_hf_med_cleaned_count <= 1, 1, inplace=True)

osa_hf_med_cleaned_count_df = osa_hf_med_cleaned_count.reset_index(name='count')

print(osa_hf_med_cleaned_count_df.shape[0])
osa_hf_med_cleaned_count_df.head()

In [ ]:
osa_hf_med_cleaned_count_df.sort_values(by = 'count', ascending = False).head()

## -- Graph used if NOT using binary value --
osa_hf_med_cleaned_count_df['count'].hist(bins = 100)

In [ ]:
# Pivot DataFrame -> icd_code as columns
df_pivot = osa_hf_med_cleaned_count_df\
    .pivot(index=['subject_id', 'hadm_id'], columns=['medication'], values='count')

osa_hf_med_cleaned_pivot = df_pivot\
    .rename_axis(columns=None)\
    .reset_index()

osa_hf_med_cleaned_pivot = osa_hf_med_cleaned_pivot\
    .merge(osa_hf_med_cleaned[['subject_id', 'hadm_id']].drop_duplicates(subset=['subject_id', 'hadm_id']), on=['subject_id', 'hadm_id'], how='outer', validate="1:1")\
    .fillna(0)\
    .astype(int)


print(osa_hf_med_cleaned_pivot.shape)
osa_hf_med_cleaned_pivot.head(3)

In [ ]:
# Medication dataframe
osa_hf_med_final = osa_hf_med_cleaned.loc[:, ['subject_id', 'hadm_id']]\
    .drop_duplicates(subset=['subject_id', 'hadm_id'])\
    .merge(osa_hf_med_cleaned_pivot, on=['subject_id', 'hadm_id'], how="left")

In [ ]:
osa_hf_med_final.head(3)

In [ ]:
## -- Using count --
# osa_hf_med_final.to_csv(root_path + "2025/cohort/osa_hf_med.csv")

## -- Using binary --
# osa_hf_med_final.to_csv(root_path + "2025/cohort/osa_hf_med_binary.csv")

#### Explore medications (using count) within cohort

In [ ]:
osa_hf_med = pd.read_csv(root_path + "2025/cohort/osa_hf_med.csv").drop(columns=['Unnamed: 0'], errors='ignore')

In [ ]:
osa_hf_med.columns

In [ ]:
# Example of medication where multiple patients have a count > 41
osa_hf_med['ALPRAZolam'].value_counts().sort_index(ascending=False).iloc[0:10]

In [ ]:
# Example of medication where multiple patients have a count > 133
osa_hf_med['sevelamer CARBONATE'].value_counts().sort_index(ascending=False).iloc[0:10]

# VI. ADD PROCEDURES

Use count of procedures per admission to represent value

In [ ]:
procedures_icd.columns

In [ ]:
d_icd_procedures.columns

In [ ]:
# merge medications (emar, emar_detail, and pharmacy)
osa_hf_procedures = data_id_only.merge(procedures_icd, how = 'left', on = ['subject_id', 'hadm_id'])
osa_hf_procedures = osa_hf_procedures.merge(d_icd_procedures, how = 'left', on = ['icd_code','icd_version'])

In [ ]:
print(osa_hf_procedures.shape)
osa_hf_procedures.head(3)

In [ ]:
# Group by subject_id and check if all icd_code entries are null
mask = osa_hf_procedures.groupby('subject_id')['icd_code'].apply(lambda x: x.isnull().all())

# Filter to only include those subject_ids
result = osa_hf_procedures[osa_hf_procedures['subject_id'].isin(mask[mask].index)]

In [ ]:
print("# of patients who had no procedures done across all admissions: ", len(result['subject_id'].unique()))
print("# of admissions who had no procedures done across all admissions: ", len(result['hadm_id'].unique()))

In [ ]:
print("Unique patients: ", osa_hf_procedures['subject_id'].unique().size)
print("Unique admissions: ", osa_hf_procedures['hadm_id'].unique().size)

In [ ]:
osa_hf_procedures.shape

<b>Store icd_code and long_title for easy conversion</b>

In [ ]:
labels_procedures = osa_hf_procedures[['icd_code', 'long_title']].drop_duplicates(subset='icd_code')
labels_procedures = pd.Series(labels_procedures['long_title'].values,index=labels_procedures['icd_code'].apply(str).values)

In [ ]:
labels_procedures.shape[0]

In [ ]:
labels_procedures.to_csv(root_path + '2025/data/labels_procedures.csv')

#### only keep certain percentage % of procedures

In [ ]:
unique_procedures = osa_hf_procedures['icd_code'].unique()
print("# of unique procedures: ", unique_procedures.size)

In [ ]:
# For each lab, calculate percentage of patients that have the lab recorded
procedure_percentage_list = []
for icd_code in tqdm(unique_procedures):
    procedure_rows = osa_hf_procedures[osa_hf_procedures['icd_code'] == icd_code]
    percentage = procedure_rows['hadm_id'].unique().size / osa_hf_procedures['hadm_id'].unique().size * 100
    procedure_percentage_list.append([icd_code, percentage])

In [ ]:
# Convert list to DataFrame
procedure_percentage_df = pd.DataFrame(procedure_percentage_list, columns=["icd_code", "percentage"])
procedure_percentage_df.sort_values(by=["percentage"], ascending=False, ignore_index=True, inplace=True)
print("Lowest percentages: ")
procedure_percentage_df.tail(10)

<b>To CSV file (procedures for OSA-HF cohort)</b>

In [ ]:
# procedure_percentage_df.to_csv(root_path + '2025/data/common_procedures.csv')

In [ ]:
procedure_percentage_df = pd.read_csv(root_path + '2025/data/common_procedures.csv', index_col=False)

n = 0.5 # customized percentage
value = procedure_percentage_df[procedure_percentage_df["percentage"] > n].shape[0]
print(f"Number of procedures in > {n} % of admissions: {value}")

#note: even .1% drops the number from 5k+ to 405

In [ ]:
procedure_percentage_df

In [ ]:
# If doing cutoff percentage value

procedure_keep = procedure_percentage_df[procedure_percentage_df["percentage"] > n]

osa_hf_procedures_cleaned = osa_hf_procedures.copy()

osa_hf_procedures_cleaned['icd_code'] = osa_hf_procedures['icd_code'].where(
    osa_hf_procedures['icd_code'].isin(procedure_keep['icd_code']), 
    other=pd.NA  # or use np.nan if preferred
)

osa_hf_procedures_cleaned['long_title'] = osa_hf_procedures['long_title'].where(
    osa_hf_procedures['icd_code'].isin(procedure_keep['icd_code']), 
    other=pd.NA  # or use np.nan if preferred
)

In [ ]:
osa_hf_procedures_cleaned.columns

In [ ]:
print("Original # of admissions:", data.shape[0])
print("Original # of patients:", data['subject_id'].unique().size, "\n")
print("# of admissions after adding procedures:", osa_hf_procedures_cleaned['hadm_id'].unique().size)
print("# of patients after adding procedures:", osa_hf_procedures_cleaned['subject_id'].unique().size)
osa_hf_procedures_cleaned.shape[0]

In [ ]:
osa_hf_procedures_cleaned['long_title'].unique().shape

In [ ]:
osa_hf_procedures_cleaned['icd_code'].unique().shape

In [ ]:
osa_hf_procedures_cleaned.head()

In [ ]:
osa_hf_procedures_cleaned_count = osa_hf_procedures_cleaned.groupby(['subject_id', 'hadm_id','icd_code'])['seq_num'].count()
osa_hf_procedures_cleaned_count_df = osa_hf_procedures_cleaned_count.reset_index(name='count')

print(osa_hf_procedures_cleaned_count_df.shape[0])
osa_hf_procedures_cleaned_count_df.head()

In [ ]:
osa_hf_procedures_cleaned_count_df.sort_values(by = 'count', ascending = False).head()
osa_hf_procedures_cleaned_count_df['count'].hist(bins = 100)

In [ ]:
# Pivot DataFrame -> icd_code as columns
df_pivot = osa_hf_procedures_cleaned_count_df\
    .pivot(index=['subject_id', 'hadm_id'], columns=['icd_code'], values='count')

# Keep all rows, regardless if they have an abnormal value recorded
osa_hf_procedures_cleaned_pivot = df_pivot\
    .rename_axis(columns=None)\
    .reset_index()

osa_hf_procedures_cleaned_pivot = osa_hf_procedures_cleaned_pivot\
    .merge(osa_hf_procedures_cleaned[['subject_id', 'hadm_id']].drop_duplicates(subset=['subject_id', 'hadm_id']), on=['subject_id', 'hadm_id'], how='outer', validate="1:1")\
    .fillna(0)\
    .astype(int)


print(osa_hf_procedures_cleaned_pivot.shape)
osa_hf_procedures_cleaned_pivot.head(3)

In [ ]:
osa_hf_procedures_cleaned

In [ ]:
# Procedure dataframe
osa_hf_procedures_final = osa_hf_procedures_cleaned.loc[:, ['subject_id', 'hadm_id']]\
    .drop_duplicates(subset=['subject_id', 'hadm_id'])\
    .merge(osa_hf_procedures_cleaned_pivot, on=['subject_id', 'hadm_id'], how="left")

In [ ]:
osa_hf_procedures_final.head()

In [ ]:
osa_hf_procedures_final.to_csv(root_path + "2025/cohort/osa_hf_procedures.csv")

# VII. COHORT FINAL MERGE

In [ ]:
## -- Using count --
# osa_hf_med = pd.read_csv(root_path + "2025/cohort/osa_hf_med.csv", index_col=0)

## -- Using binary --
osa_hf_med = pd.read_csv(root_path + "2025/cohort/osa_hf_med_binary.csv", index_col=0)

osa_hf_procedure = pd.read_csv(root_path + "2025/cohort/osa_hf_procedures.csv", index_col=0)
osa_hf_demo_lab_diagnosis = pd.read_csv(root_path + "2025/cohort/osa_hf_all_features_labeled.csv")

In [ ]:
osa_hf_med['Acetaminophen_med_count'] = osa_hf_med['Acetaminophen']
osa_hf_med['Digoxin_med_count'] = osa_hf_med['Digoxin']
osa_hf_med['Vancomycin_med_count'] = osa_hf_med['Vancomycin']
osa_hf_med.drop(columns=['Acetaminophen', 'Digoxin', 'Vancomycin'], inplace=True)

In [ ]:
data_medications_procedures = data_id_only.merge(osa_hf_med, how = 'left', on = ['subject_id', 'hadm_id'])
data_medications_procedures = data_medications_procedures.merge(osa_hf_procedure, how = 'left', on = ['subject_id', 'hadm_id'])

In [ ]:
data_medications_procedures.drop(columns=['emar_id', 'poe_id', 'event_txt', 'enter_provider_id'], inplace=True, errors='ignore')

In [ ]:
print("# Unique patients: ", data_medications_procedures['subject_id'].unique().size)
print("# Unique admissions: ", data_medications_procedures['hadm_id'].unique().size)

In [ ]:
data_medications_procedures.shape

In [ ]:
data_medications_procedures.columns()

In [ ]:
data_medications_procedures.head(3)

In [ ]:
osa_hf_demo_lab_diagnosis_med_procedure = osa_hf_demo_lab_diagnosis.merge(data_medications_procedures, how = 'left', on = ['subject_id', 'hadm_id'])

In [ ]:
print("# Unique patients: ", osa_hf_demo_lab_diagnosis_med_procedure['subject_id'].unique().size)
print("# Unique admissions: ", osa_hf_demo_lab_diagnosis_med_procedure['hadm_id'].unique().size)

In [ ]:
# -- Checks for abnormal columns.
for column in osa_hf_demo_lab_diagnosis_med_procedure:
    if column.endswith("_y") | column.endswith("_x"):
        print(column)

In [ ]:
## -- Using medication count --
# osa_hf_demo_lab_diagnosis_med_procedure.to_csv(root_path + "2025/cohort/osa_hf_demo_lab_diagnosis_med_procedure.csv")

## -- Using binary value for medications --
# osa_hf_demo_lab_diagnosis_med_procedure.to_csv(root_path + "2025/cohort/osa_hf_demo_lab_diagnosis_medbinary_procedure.csv")

#### Processed data

In [ ]:
X = osa_hf_demo_lab_diagnosis_med_procedure.copy()

In [ ]:
X.columns

In [ ]:
labels = X['group']
print(labels.value_counts())

In [ ]:
X_labs = X.iloc[:, 16:233]

In [ ]:
X_cleaned = X.drop(columns=['osa', 'hf', 'group_name', 'text'])

In [ ]:
cols_to_normalize = ['age', 'los', 'charlson', 'bmi']
cols_to_normalize.extend(list(X_labs.columns))
cols_to_ohe = ['gender', 'race_classified', 'insurance', 'admission_type']

In [ ]:
X_cleaned[cols_to_normalize].shape

In [ ]:
print(X_cleaned[cols_to_normalize].isna().sum().sum())  # Check NaNs
print((X_cleaned[cols_to_normalize] < -1).sum().sum())  # Check negatives

In [ ]:
X_cleaned[cols_to_normalize] = np.log1p(X_cleaned[cols_to_normalize]) 
scaler = MinMaxScaler() 
X_cleaned[cols_to_normalize] = scaler.fit_transform(X_cleaned[cols_to_normalize])

In [ ]:
X_processed_with_id = one_hot_encode(X_cleaned, cols_to_ohe, drop_id=False)

In [ ]:
X_processed_with_id.head(5)

In [ ]:
X_processed_without_id = one_hot_encode(X_cleaned, cols_to_ohe, drop_id=True)

In [ ]:
X_processed_without_id.head(5)

In [ ]:
## -- Using medication count --
# X_processed_with_id.to_csv(root_path + "2025/cohort/osa_hf_demo_lab_diagnosis_med_procedure_processed_with_id.csv", index=False)

## -- Using binary value for medication
# X_processed_with_id.to_csv(root_path + "2025/cohort/osa_hf_demo_lab_diagnosis_medbinary_procedure_processed_with_id.csv", index=False)

In [ ]:
## -- Using medication count --
# X_processed_without_id.to_csv(root_path + "2025/cohort/osa_hf_demo_lab_diagnosis_med_procedure_processed_without_id.csv", index=False)

## -- Using binary value for medication
# X_processed_without_id.to_csv(root_path + "2025/cohort/osa_hf_demo_lab_diagnosis_medbinary_procedure_processed_without_id.csv", index=False)

# VIII. REMOVE FEATURES WITH LOW VARIANCE
**Removed features with < 1% variance.**

1155 features --> 576 features

In [ ]:
root_path = "/global/cfs/projectdirs/m1532/Projects_MVP/Sophia/"

In [ ]:
X, y = read_cohort_with_id(root_path + "2025/cohort/osa_hf_demo_lab_diagnosis_medbinary_procedure_processed_with_id.csv")
X_scaled, y = read_cohort_without_id(root_path + "2025/cohort/osa_hf_demo_lab_diagnosis_medbinary_procedure_processed_with_id.csv")

In [ ]:
X_raw, y_raw = read_cohort_with_id(root_path + "2025/cohort/osa_hf_demo_lab_diagnosis_medbinary_procedure_reduced.csv")

In [ ]:
X_raw.head(2)

In [ ]:
X_scaled.shape

In [ ]:
# sparsity:
sparsity = sum((X_scaled == 0).astype(int).sum())/X_scaled.size
print(sparsity)

In [ ]:
# Calculate variance of each column
variances = X_scaled.var()
print(variances)

In [ ]:
count=0
for feature, variance in variances.items():
    if (variance >= 0.01):
        count = count+1
print("Number of features after removing those with a variance of less than 1%: ", count)

In [ ]:
## -- With ID --
kept_cols = ['subject_id', 'hadm_id', 'group', 'group_name']

# Set a threshold (e.g., 10)
threshold = .01

# Initialize and fit the VarianceThreshold selector
selector = VarianceThreshold(threshold=threshold)
selector.fit(X.drop(columns=kept_cols))

# Get the variances of the features
feature_variances = selector.variances_
print("Feature Variances:", feature_variances)

# Get the indices of the selected (non-removed) features
selected_feature_indices = selector.get_support(indices=True)
# print("Selected Feature Indices:", selected_feature_indices)

# Transform the DataFrame to keep only the selected features
X_scaled_transformed_with_id = X_scaled.iloc[:, selected_feature_indices]
X_scaled_transformed_with_id = pd.concat([X[kept_cols], X_scaled_transformed_with_id], axis=1)
print("Transformed DataFrame:")
X_scaled_transformed_with_id.head(3)

In [ ]:
## -- Without ID -- 

# Set a threshold (e.g., 10)
threshold = .01

# Initialize and fit the VarianceThreshold selector
selector = VarianceThreshold(threshold=threshold)
selector.fit(X_scaled)

# Get the variances of the features
feature_variances = selector.variances_
print("Feature Variances:", feature_variances)

# Get the indices of the selected (non-removed) features
selected_feature_indices = selector.get_support(indices=True)
# print("Selected Feature Indices:", selected_feature_indices)

# Transform the DataFrame to keep only the selected features
X_scaled_transformed = X_scaled.iloc[:, selected_feature_indices]
print("Transformed DataFrame:")
X_scaled_transformed.head(3)

In [ ]:
# Merge based on index
X_scaled_transformed = pd.merge(y, X_scaled_transformed, left_index=True, right_index=True, how='inner')

In [ ]:
X_scaled_transformed_with_id.to_csv(root_path + "2025/cohort/osa_hf_demo_lab_diagnosis_medbinary_procedure_reduced_with_id.csv")
X_scaled_transformed.to_csv(root_path + "2025/cohort/osa_hf_demo_lab_diagnosis_medbinary_procedure_reduced_without_id.csv")

In [ ]:
X_scaled_transformed.to_csv("cleaned_inputs.csv", index=False)


### Among those that are kept, which are labs, diagnoses, etc. ?

In [ ]:
X_scaled_transformed.columns

In [ ]:
medications = pd.read_csv(root_path + "2025/data/common_medications.csv", index_col=0)['medication'].tolist()
procedures = pd.read_csv(root_path + "2025/data/common_procedures.csv", index_col=0)['icd_code'].tolist()
diagnoses = pd.read_csv(root_path + "2025/data/labels_icd_code_filtered.csv", index_col=0)['long_title'].tolist()
labs = pd.read_csv(root_path + "2025/data/labels_labevents_filtered.csv", index_col=0)['label'].tolist()

In [ ]:
# -- Using reduced feature space (n=666)
kept_medications = []
kept_procedures = []
kept_diagnoses = []
kept_labs = []
kept_demographics = []

for column in X_scaled_transformed:
    # Handle columns with suffix
    suffix = [".1", ".2", ".3", ".4", ".5"]

    for idx, string in enumerate(suffix):
        if column.endswith(suffix[idx]):
            column = column[:-len(suffix[idx])]
    
    if (column in medications) | ("_med_count" in column):
        kept_medications.append(column)
    elif column in procedures:
        kept_procedures.append(column)
    elif column in diagnoses:
        kept_diagnoses.append(column)
    elif column in labs:
        kept_labs.append(column)
    else:
        kept_demographics.append(column)

print("Number of medication features (reduced): ", len(kept_medications))
print("Number of procedure features (reduced): ", len(kept_procedures))
print("Number of diagnosis features (reduced): ", len(kept_diagnoses))
print("Number of lab features (reduced): ", len(kept_labs))
print("Number of demographic features (reduced): ", len(kept_demographics))
print("Total number of features (reduced): ", len(kept_medications) + len(kept_procedures) + len(kept_diagnoses) + len(kept_labs) + len(kept_demographics))

In [ ]:
kept_demographics

In [ ]:
# -- Using all feature space (n=1156)
all_medications = []
all_procedures = []
all_diagnoses = []
all_labs = []
all_demographics = []

for column in X_scaled:
    # Handle columns with suffix
    suffix = [".1", ".2", ".3", ".4", ".5"]

    for idx, string in enumerate(suffix):
        if column.endswith(suffix[idx]):
            column = column[:-len(suffix[idx])]
    
    if (column in medications) | ("_med_count" in column):
        all_medications.append(column)
    elif column in procedures:
        all_procedures.append(column)
    elif column in diagnoses:
        all_diagnoses.append(column)
    elif column in labs:
        all_labs.append(column)
    else:
        all_demographics.append(column)

print("Number of medication features (all): ", len(all_medications))
print("Number of procedure features (all): ", len(all_procedures))
print("Number of diagnosis features (all): ", len(all_diagnoses))
print("Number of lab features (all): ", len(all_labs))
print("Number of demographic features (all): ", len(all_demographics))
print("Total number of features (all): ", len(all_medications) + len(all_procedures) + len(all_diagnoses) + len(all_labs) + len(all_demographics))

In [ ]:
print("[Medication] # of removed features: ", len(all_medications) - len(kept_medications))
print("[Procedure] # of removed features: ", len(all_procedures) - len(kept_procedures))
print("[Diagnosis] # of removed features: ", len(all_diagnoses) - len(kept_diagnoses))
print("[Lab] # of removed features: ", len(all_labs) - len(kept_labs))
print("[Demographic] # of removed features: ", len(all_demographics) - len(kept_demographics))

# VIII. LATENT SPACE EXTRACTION
Using Autoencoder (50 dimensions) -> UMAP (2 dimensions)

** Prior to this, conduct a grid search with cohort on all hyperparameters for Autoencoder architecture and UMAP (separately)

## AE-UMAP on reduced feature data

In [ ]:
X_scaled_transformed, y = read_cohort_without_id(root_path + "2025/cohort/osa_hf_demo_lab_diagnosis_medbinary_procedure_reduced.csv")

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X_scaled_transformed, y, test_size = 0.3, random_state = 42)
# print(y_train.value_counts())
# print(y_test.value_counts())

#### Autoencoder

In [ ]:
# # Set the seed
# set_seed(42)

# # Check if CUDA (GPU) is available
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# #_ for sparse matrix data type only
# # X_train_dense = csr_matrix(X_train).toarray() 
# # X_test_dense = csr_matrix(X_test).toarray()
# # X_train_tensor = torch.tensor(X_train_dense, dtype=torch.float32).to(device)
# # X_test_tensor = torch.tensor(X_test_dense, dtype=torch.float32).to(device)

# X_train_np = X_train.to_numpy()
# X_test_np = X_test.to_numpy()
# X_scaled_np = X_scaled_transformed.to_numpy()

# X_train_tensor = torch.tensor(X_train_np, dtype=torch.float32).to(device)
# X_test_tensor = torch.tensor(X_test_np, dtype=torch.float32).to(device)

# #- all data points
# #X_normalized_dense = X_normalized.toarray()
# X_tensor = torch.tensor(X_scaled_np, dtype = torch.float32).to(device)

# # Define the model
# input_dim = X_train_np.shape[1]  # Example input dimension
# encoding_dims = 50  # Example encoding dimension
# model = Autoencoder(input_dim, encoding_dims).to(device)

# # Define the optimizer and loss function
# optimizer = optim.Adam(model.parameters(), lr=0.001)  # Using Adam optimizer
# criterion = nn.MSELoss()  # Mean Squared Error Loss

# # Convert data into PyTorch DataLoader and move to GPU
# batch_size = 32

# X_dataset = TensorDataset(X_train_tensor, X_train_tensor)  # Using X_train_tensor for both input and target
# X_loader = DataLoader(X_dataset, batch_size=batch_size, shuffle=True)

# start_time = time.time()
# # Example training loop
# num_epochs = 20
# for epoch in range(num_epochs):
#     for data in X_loader:   
#         #- AE
#         inputs, targets = data[0].to(device), data[1].to(device)  # Move batch to GPU
#         outputs = model(inputs)        
#         #- DAE
#         #inputs, targets = model.add_noise(data[0]).to(device), data[1].to(device)  # Move batch to GPU
#         #outputs = model(inputs)             
#         loss = criterion(outputs, targets)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        
#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# model.eval() 
# with torch.no_grad():
#     #latent_representations = model.encode(X_test_tensor).cpu().numpy()
#     latent_representations = model.encode(X_tensor).cpu().numpy()
    
# end_time = time.time()
# print("AE Runtime: ", end_time - start_time)

In [ ]:
#- use raw features
# train_models_results(latent_representations,Y)

#- umap now for plotting

import umap.umap_ as umap
clusterable_embedding_ae, runtime = umap_embedding(latent_representations, n_components = 2)

In [ ]:
labels = y
group = list(labels.unique())
plot_umap_embedding(clusterable_embedding_ae, labels, group, colors=['orange', 'red', 'blue'])

In [ ]:
type(latent_representations)

In [ ]:
type(clusterable_embedding_ae)

In [ ]:
# # Save latent representation and embeddings to file
# np.save(root_path + "2025/data/latent_representations.npy", latent_representations)
# np.save(root_path + "2025/data/clusterable_embedding_ae.npy", clusterable_embedding_ae)

# IX. CLUSTERING
Using Spectral Clustering

# X. FEATURE IMPORTANCE
Using Logistic Regression and Shapley analysis

# XI. RISK ANALYSIS
Using Kaplan-Meier (K-M) survival analysis